In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import LeaveOneOut, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.utils import resample
from procan_connectome.config import DATA_PATH, RANDOM_STATE
from procan_connectome.data_processing.linear_svc_importance_filter import LinearSVCImportanceFilter
from procan_connectome.data_processing.rf_importance_filter import RFImportanceFilter
from procan_connectome.eda.EDA import rf_loovc
from procan_connectome.data_processing.powertransformer_wrapper import PowerTransformerWrapper
import logging
import datetime

In [3]:
if __name__ == "__main__": 
    df = pd.read_csv(os.path.join(DATA_PATH, "combined_datasets_global.csv"))
    df = df.set_index('ID')
    df

,fun-HeadMovement,label,fun-Eg,fun-Eloc,fun-Gamma,fun-Lambda,fun-Cp,fun-Sigma,fun-Lp,fun-bzscore,...,str-Gamma,str-Lambda,str-Cp,str-Sigma,str-Lp,str-bzscore,str-rzscore,str-szscore,str-Connectome_density,str-Connectome_intensity
ID,,,,,,,,,,,,,,,,,,,,,
CAL_004,0.045966,Stage_0,0.663254,0.830872,1.749309,1.025427,0.666953,1.705932,1.507717,-4.810287,...,1.329800,1.009632,0.670522,1.317114,1.419374,6.026401,11.805093,-29.673321,0.4291,0.4427
CAL_005,0.047396,Stage_1b,0.620641,0.791283,1.900540,1.031656,0.595890,1.842222,1.611238,-2.330884,...,1.255270,1.003088,0.647748,1.251406,1.397743,3.390472,4.820435,-1.487338,0.4422,0.4294
CAL_007,0.078991,Stage_1b,0.630087,0.774496,1.823195,1.024414,0.559852,1.779744,1.587081,0.800517,...,1.285756,1.006716,0.656097,1.277179,1.427977,4.464224,8.890078,-1.319985,0.4170,0.4289
CAL_008,0.077669,Stage_1a,0.641698,0.794428,1.830358,1.021718,0.594487,1.791452,1.558366,0.274330,...,1.195574,1.000625,0.655978,1.194827,1.377630,4.271875,8.274111,-21.729664,0.4588,0.4626
CAL_009,0.082430,Stage_1b,0.637536,0.811208,1.786458,1.035106,0.630358,1.725870,1.568538,-4.353755,...,1.253163,1.003818,0.642060,1.248397,1.409798,2.866312,6.803213,-3.018963,0.4309,0.4523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SB_040,0.096126,Stage_1a,0.663754,0.789021,1.400942,1.013446,0.584322,1.382354,1.506583,-3.402909,...,1.186515,1.003735,0.675412,1.182100,1.369621,4.535404,9.635594,-13.498499,0.4719,0.4398
SB_041,0.087158,Stage_1b,0.634207,0.788663,1.891594,1.021895,0.583268,1.851065,1.576772,3.270413,...,1.229554,1.006067,0.659854,1.222139,1.410874,3.682689,9.675177,-9.014062,0.4328,0.4889
SB_042,0.064763,Stage_0,0.646608,0.804616,1.546333,1.022312,0.619791,1.512584,1.546531,-4.186258,...,1.253016,1.004535,0.651616,1.247359,1.411785,5.822824,7.877808,-12.820783,0.4296,0.4454


In [4]:
if __name__ == "__main__": 
    X, y = df.drop(columns='label'), df['label']
    pt = PowerTransformerWrapper() 
    X_trans = pt.fit_transform(X)
    X_trans

,fun-HeadMovement,fun-Eg,fun-Eloc,fun-Gamma,fun-Lambda,fun-Cp,fun-Sigma,fun-Lp,fun-bzscore,fun-rzscore,...,str-Gamma,str-Lambda,str-Cp,str-Sigma,str-Lp,str-bzscore,str-rzscore,str-szscore,str-Connectome_density,str-Connectome_intensity
ID,,,,,,,,,,,,,,,,,,,,,
CAL_004,-0.604655,1.278294,1.344323,-0.304018,0.186169,1.324641,-0.337403,-1.276160,-1.433862,-0.695674,...,0.000932,0.647195,1.155415,-0.044702,-0.338838,0.426052,1.875456,-2.028669,0.491316,-0.270783
CAL_005,-0.533050,-1.679453,-0.273312,0.886039,0.957441,-0.128602,0.833760,1.679328,-0.519928,0.245327,...,-0.637250,-0.647195,0.536508,-0.627896,-0.749527,-1.321790,-0.186042,1.499337,0.754826,-0.736186
CAL_007,0.654940,-0.847361,-1.164696,0.237967,0.053191,-1.113286,0.254762,0.845801,0.664068,-0.324206,...,-0.361939,0.323597,0.779615,-0.388239,-0.184038,-0.622211,1.194064,1.550534,0.250266,-0.752660
CAL_008,0.617589,0.022155,-0.121594,0.294383,-0.319147,-0.163350,0.357559,-0.022274,0.460884,-1.689981,...,-1.241659,-0.970792,0.776287,-1.209489,-1.160896,-0.748904,1.025352,-1.366011,1.092535,0.532780
CAL_009,0.748087,-0.271784,0.615855,-0.040433,1.356374,0.646054,-0.186598,0.270961,-1.266437,1.293208,...,-0.657063,-0.323597,0.359017,-0.656854,-0.516773,-1.655891,0.574076,1.098457,0.527367,0.099841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SB_040,1.067600,1.302788,-0.385230,-2.056689,-1.648926,-0.423346,-2.035674,-1.300677,-0.916589,0.919827,...,-1.341718,-0.323597,1.271235,-1.351830,-1.333153,-0.575184,1.385173,-0.528196,1.362038,-0.376865
SB_041,0.867214,-0.520871,-0.403194,0.806576,-0.292551,-0.451133,0.921973,0.519599,1.632984,-0.990899,...,-0.886426,0.000000,0.882677,-0.918712,-0.496477,-1.133317,1.394954,0.042759,0.565475,1.825381
SB_042,0.202543,0.345697,0.339990,-1.464850,-0.239360,0.423891,-1.500126,-0.344944,-1.204926,0.999181,...,-0.658451,-0.323597,0.651621,-0.666888,-0.479339,0.287638,0.910981,-0.448760,0.501325,-0.169610


In [7]:
if __name__ == "__main__": 
    svc = LinearSVC(random_state=RANDOM_STATE, max_iter = 100000000)
    svcif = LinearSVCImportanceFilter(random_state=RANDOM_STATE, threshold=0.001, sort=False)
    svcif.fit(X_trans, y)

/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVCImportanceFilter(sort=False, threshold=0.001)

In [8]:
def svc_loovc(X: pd.DataFrame, y: pd.Series, standard_scale: bool=False, power_transform:bool=False): 
  loo = LeaveOneOut()
  importances = []
  y_true = []
  y_predict = []
  counter = 0

  for train_idx, test_idx in loo.split(X):
    counter += 1 
    if counter % 10 == 0: 
      print(f"Iteration {counter} of {len(X)}")
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y[train_idx], y[test_idx]
    if standard_scale: 
      std_scaler = StandardScaler()
      X_train = std_scaler.fit_transform(X_train)
      X_test = std_scaler.transform(X_test)
    if power_transform: 
      pt = PowerTransformer()
      X_train = pt.fit_transform(X_train)
      X_test = pt.transform(X_test)
    svc = LinearSVCImportanceFilter(random_state=RANDOM_STATE, sort=False)
    svc.fit(X_train,y_train)
    importances.append(svc.feature_importances_df_['Importance'].values)
    y_predict.append(svc.estimator_.predict(X_test)[0])
    y_true.append(y_test[0])

  acc = accuracy_score(y_true, y_predict)
  logging.debug(f"Feature Selection LOOCV Accuracy: {acc}")
  labels = X.columns
  feature_importances = np.array(importances).mean(axis=0)
  feature_importances_df = pd.DataFrame(list(zip(map(lambda x: round(x,6), feature_importances), labels)), columns=["Importance", "Feature"])
  logging.debug(f"First 10 Features: {feature_importances_df.iloc[:10]}")
  results_df = pd.DataFrame({
      "y_true": y_true,
      'y_pred': y_predict
  }).set_index(X.index)
  return results_df, feature_importances_df

if __name__ == "__main__": 
  X, y = df.drop(columns="label"), df['label']
  results_df, feature_importances_df = svc_loovc(X_trans, y, False, False)
  feature_importances_df.to_csv(os.path.join(DATA_PATH, 'SVC_feature_importances_global.csv'), index=False)

/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: L

Iteration 10 of 171


/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: L

Iteration 20 of 171


/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: L

...

Iteration 160 of 171


/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: L

Iteration 170 of 171
Accuracy: 0.3216374269005848
Features sorted by score:


/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mklasby/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,Importance,Feature
0,0.091362,fun-Connectome_density
1,0.074714,fun-Eloc
2,0.069199,str-Connectome_density
3,0.055040,str-Cp
4,0.049777,fun-Cp
5,0.048142,fun-Lambda
6,0.045160,str-Gamma
7,0.041351,Site
8,0.038471,str-Lambda
9,0.037304,fun-Lp


In [18]:
if __name__ == "__main__": 
  X, y = df.drop(columns="label"), df['label']
  results_df, feature_importances_df = rf_loovc(X_trans, y, False, False)
  feature_importances_df.to_csv(os.path.join(DATA_PATH, 'rf_feature_importances_2.csv'), index=False)

Accuracy: 0.3157894736842105
Features sorted by score:


,Importance,Feature
0,0.003181,str-CustomPc_normalized-Temporal Pole Mid R
1,0.002995,str-NCp-Cuneus L
2,0.002698,fun-NCp-Hippocampus L
3,0.002344,str-NLe-Parietal Inf R
4,0.002287,fun-Bc-Postcentral R
...,...,...
1354,0.000229,fun-Dc-Rolandic Oper L
1355,0.000219,str-Dc-Occipital Mid L
1356,0.000195,str-SumEdgeNum_Within_Module04
1357,0.000184,str-SumEdgeNum_Between_Module03_04
